In [1]:
from github import Github, GithubException
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# ----------- Saisie sécurisée du token GitHub ----------- #
token_input = widgets.Password(description="Token GitHub:")
username_input = widgets.Text(description="Utilisateur:")

display(widgets.VBox([token_input, username_input]))

connect_button = widgets.Button(description="Se connecter", button_style='success')
output_connect = widgets.Output()

# ----------- Interface principale après connexion ----------- #
def display_repo_explorer(g):
    user = g.get_user()
    repos = list(user.get_repos())

    repo_dropdown = widgets.Dropdown(
        options=[(repo.full_name, repo.full_name) for repo in repos],
        description="Dépôt:",
        layout=widgets.Layout(width='600px')
    )

    explore_button = widgets.Button(description="Explorer le dépôt", button_style='primary')
    file_dropdown = widgets.Dropdown(description="Fichier:", layout=widgets.Layout(width='600px'))
    output_explore = widgets.Output()
    output_file_content = widgets.Output()  # ⬅️ Zone dédiée au contenu du fichier

    def get_all_files(repo, path="", ref="main", files=None):
        if files is None:
            files = []
        try:
            contents = repo.get_contents(path, ref)
            for content in contents:
                if content.type == "dir":
                    get_all_files(repo, content.path, ref, files)
                elif content.type == "file":
                    files.append(content.path)
        except GithubException as e:
            print(f"⚠️ Erreur d'accès à {path} : {e.data.get('message', 'Inconnue')}")
        return files

    def explore_repo(_):
        with output_explore:
            clear_output()
            try:
                repo = g.get_repo(repo_dropdown.value)
                branch = repo.default_branch
                display(Markdown(f"### 📦 Dépôt : `{repo.full_name}` — Branche : `{branch}`"))
                all_files = get_all_files(repo, "", branch)
                if not all_files:
                    print("Ce dépôt ne contient aucun fichier.")
                file_dropdown.options = all_files
            except GithubException as e:
                print(f"❌ Erreur : {e.data.get('message', 'Inconnue')}")

    def on_file_selected(change):
        with output_file_content:
            clear_output(wait=True)
            try:
                repo = g.get_repo(repo_dropdown.value)
                branch = repo.default_branch
                file_path = change['new']
                file_content = repo.get_contents(file_path, branch)
                decoded = file_content.decoded_content.decode("utf-8")

                display(Markdown(f"### 📄 Contenu de `{file_path}`"))
                print(f"🧾 Taille : {file_content.size} octets\n")
                if file_path.endswith(".md"):
                    display(Markdown(decoded))
                else:
                    print(decoded)
            except GithubException as e:
                print(f"⚠️ Erreur lors de l'accès à {file_path} : {e.data.get('message', 'Inconnue')}")

    explore_button.on_click(explore_repo)
    file_dropdown.observe(on_file_selected, names='value')

    display(widgets.VBox([
        repo_dropdown,
        explore_button,
        output_explore,
        file_dropdown,           # ⬅️ Toujours visible
        output_file_content      # ⬅️ Zone dédiée au contenu
    ]))

# ----------- Connexion GitHub ----------- #
def connect_to_github(_):
    output_connect.clear_output()
    with output_connect:
        try:
            g = Github(token_input.value)
            user = g.get_user()
            if user.login.lower() != username_input.value.lower():
                print(f"❌ Token valide mais associé à : {user.login}")
            else:
                print(f"✅ Connecté avec succès : {user.login}")
                display_repo_explorer(g)
        except GithubException as e:
            print(f"Erreur d'authentification : {e.data.get('message', 'Inconnue')}")

connect_button.on_click(connect_to_github)
display(connect_button, output_connect)

Button(button_style='success', description='Se connecter', style=ButtonStyle())

Output()